# Extracción de datos

In [54]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [26]:
# Esquema de tipos para leer el CSV procesado
dtype_dict = {
    'sucursal': 'category',
    'producto_id': 'category',
    'unidades_vendidas': 'int16',
    'precio_venta': 'float32',
    'unidades_compradas': 'int16',
    'costo_unitario': 'float32',
    'margen_bruto': 'float32',
    'nombre_producto': 'category',
    'categoria': 'category',
    'marca': 'category',
    'unidad_medida': 'category',
    'tipo_promocion': 'category',
    'descuento_porcentual': 'float32',
    'producto_en_promocion': 'int8'
}

# Cargar el DataFrame y aplicar el esquema de tipos
df = pd.read_csv(
    '../data/processed/tabla_consolidada.csv',
    parse_dates=['fecha'],
    dtype=dtype_dict,
    low_memory=False
)


# Análisis estratégico

## Top productos con mayor rotación por sucursal en el último semestre

In [27]:
# Fecha de corte para el semestre
fecha_limite = df['fecha'].max() - pd.DateOffset(months=6)

# Top 10 productos por sucursal en el último semestre
top10_rotacion = (
    df[df['fecha'] >= fecha_limite]
    .groupby(['sucursal', 'producto_id'], as_index=False, observed=True)['unidades_vendidas']
    .sum()
    .query('unidades_vendidas > 0')
    .sort_values(['sucursal', 'unidades_vendidas'], ascending=[True, False])
    .groupby('sucursal', observed=True)
    .head(10)
    .merge(df[['producto_id', 'nombre_producto', 'categoria']].drop_duplicates(), on='producto_id', how='left')
)


In [28]:
top10_rotacion

,sucursal,producto_id,unidades_vendidas,nombre_producto,categoria
0,Sucursal Este,P009,43,Producto 9,Lácteos
1,Sucursal Este,P020,41,Producto 20,Limpieza
2,Sucursal Este,P019,19,Producto 19,Lácteos
3,Sucursal Este,P007,5,Producto 7,Bebidas
4,Sucursal Este,P015,3,Producto 15,Bebidas
5,Sucursal Norte,P003,46,Producto 3,Limpieza
6,Sucursal Norte,P013,41,Producto 13,Snacks
7,Sucursal Norte,P017,35,Producto 17,Lácteos
8,Sucursal Norte,P007,31,Producto 7,Bebidas
9,Sucursal Norte,P001,29,Producto 1,Bebidas


### Análisis:

El análisis de rotación de productos por sucursal durante el último semestre revela patrones consistentes de consumo que pueden guiar decisiones estratégicas de abastecimiento y promociones.

Lácteos y Bebidas dominan las listas en todas las sucursales, lo cual sugiere una alta frecuencia de consumo y necesidad de mantener inventarios estables en estas categorías.

Productos como "Producto 9" (Lácteos) y "Producto 3" (Limpieza) destacan en más de una sucursal, indicando que tienen potencial para negociaciones centralizadas con proveedores o para campañas de promoción regionales.

Algunas sucursales presentan productos de bajo volumen dentro del top (por ejemplo, Producto 4 con solo 1 unidad en Sucursal Oeste), lo cual puede indicar baja rotación general en esa categoría o surtido ineficiente.

Este análisis puede usarse como punto de partida para revisar el mix de productos por zona, reforzar el abastecimiento de productos de alta demanda y revisar la rentabilidad de los más vendidos.

## ~~Quiebres de inventario~~

Análisis descartado para interpretación y toma estratégica de decisiones. Se mantiene la lógica que lleva a esa decisión:

In [55]:
# Paso 1: promedio histórico por producto y sucursal
promedio_ventas = (
    df.groupby(['sucursal', 'producto_id'], observed=True)['unidades_vendidas']
    .mean()
    .reset_index()
    .rename(columns={'unidades_vendidas': 'promedio_historico'})
)

# Paso 2: merge + detección de quiebres
quiebres = (
    df.merge(promedio_ventas, on=['sucursal', 'producto_id'], how='left')
    .query('unidades_vendidas == 0 and promedio_historico > 0.5')
    .copy()
)

# Paso 3: crea una columna bandera (opcional si vas a usar en dashboard)
quiebres['es_quiebre'] = True


Lo anterior genera un DataFrame vacío al correr `print(quiebres)`. Por ello, se evalúan las posibles razones subyacentes

In [56]:
promedio_ventas['promedio_historico'].describe()


count    80.000000
mean      0.038776
std       0.043973
min       0.000000
25%       0.000000
50%       0.031746
75%       0.054705
max       0.188209
Name: promedio_historico, dtype: float64

Vemos que la media de `promedio_histórico` es inferior a la condición propuesta de `promedio_historico > 0.5`. Por ello, se procede a reevaluar esta condición con base en el cálculo de un umbral dinámico.

In [ ]:
umbral_dinamico = promedio_ventas['promedio_historico'].quantile(0.50) # 
# print(umbral_dinamico) # El resultado es 0.0317, lo que refleja proximadamente una venta cada 32 días por producto y sucursal

np.float64(0.031746031746031744)

Sabiendo esto, se opta por evaluar productos que vendan al menos una vez cada 7 días, así:  
umbral = 1 / 7  ≈ 0.14


In [62]:

quiebres = (
    df.merge(promedio_ventas, on=['sucursal', 'producto_id'], how='left')
    .query(f'unidades_vendidas == 0 and promedio_historico > 0.14')
    .copy()
)


In [63]:
print(f"Número de quiebres detectados: {len(quiebres)}")


Número de quiebres detectados: 2632


In [64]:
quiebres[['fecha', 'sucursal', 'producto_id', 'promedio_historico']].head()


,fecha,sucursal,producto_id,promedio_historico
0,2023-01-01,Sucursal Oeste,P016,0.188209
18,2023-01-01,Sucursal Oeste,P011,0.183673
32,2023-01-01,Sucursal Norte,P003,0.143991
80,2023-01-02,Sucursal Oeste,P016,0.188209
98,2023-01-02,Sucursal Oeste,P011,0.183673


Dada la naturaleza de los datos, con pocos registros de ventas para un lapso prolongado de tiempo, se opta por realizar un análisis semanal en vez de diario:

In [65]:
df['semana'] = df['fecha'].dt.to_period('W').apply(lambda r: r.start_time)

ventas_semanales = (
    df.groupby(['semana', 'sucursal', 'producto_id'], observed=True)['unidades_vendidas']
    .sum()
    .reset_index()
)

promedios_semanales = (
    ventas_semanales
    .groupby(['sucursal', 'producto_id'], observed=True)['unidades_vendidas']
    .mean()
    .reset_index()
    .rename(columns={'unidades_vendidas': 'promedio_semanal'})
)

quiebres_semanales = (
    ventas_semanales
    .merge(promedios_semanales, on=['sucursal', 'producto_id'], how='left')
    .query('unidades_vendidas == 0 and promedio_semanal > 0.14')  # o ajustable
)

print(f"Número de quiebres semanales detectados: {len(quiebres_semanales)}")

Número de quiebres semanales detectados: 5501


### Se decide detener aquí la profundización en este punto:

#### Realidad de la data

Supongamos que tenemos:

- ~2 años = ~104 semanas  
- 10 sucursales  
- 50 productos  

→ Posibles combinaciones:  
`104 semanas × 10 sucursales × 50 productos = 52,000` combinaciones semana–producto–sucursal

De esas 52,000 combinaciones:

- Solo hay ~100 registros reales de venta  
- Por tanto, ~51,900 combinaciones no tuvieron venta esa semana

Y si el promedio semanal de un producto-sucursal supera **0.14** (≈ 1 venta cada 7 semanas),  
entonces **casi cualquier semana sin venta podría ser considerada como quiebre**,  
aunque en realidad no haya una demanda sostenida.

**Conclusión**: El análisis de quiebres pierde validez en este contexto debido a la baja densidad de datos. El volumen de quiebres está inflado por escasez de datos, no porque realmente haya ocurrido una ruptura de inventario.


## Análisis de proveedores

In [77]:
# Cargar el DataFrame de proveedores
file_path = '../data/raw/base_datos_colombia.xlsx'
df_proveedores = pd.read_excel(file_path, sheet_name='proveedores', engine='openpyxl')


### 1. Z-Score

In [ ]:
# Análisis por z-score
df_proveedores['z_calificacion'] = zscore(df_proveedores['calificacion_proveedor'])
df_proveedores['z_plazo'] = zscore(df_proveedores['plazo_entrega_dias'])

# Proveedores con calificación muy baja (z < -1)
baja_calif_z = df_proveedores[df_proveedores['z_calificacion'] < -1]

# Proveedores con plazos muy largos (z > 1)
plazos_largos_z = df_proveedores[df_proveedores['z_plazo'] > 1]

### 2. Análisis por quantiles

In [79]:
# Análisis por quantiles
# Calificación
q1_calif = df_proveedores['calificacion_proveedor'].quantile(0.25)
q3_calif = df_proveedores['calificacion_proveedor'].quantile(0.75)
iqr_calif = q3_calif - q1_calif
baja_calif_iqr = df_proveedores[df_proveedores['calificacion_proveedor'] < (q1_calif - 1.5 * iqr_calif)]

# Plazos
q1_plazo = df_proveedores['plazo_entrega_dias'].quantile(0.25)
q3_plazo = df_proveedores['plazo_entrega_dias'].quantile(0.75)
iqr_plazo = q3_plazo - q1_plazo
plazos_largos_iqr = df_proveedores[df_proveedores['plazo_entrega_dias'] > (q3_plazo + 1.5 * iqr_plazo)]


In [82]:
df_proveedores['riesgo_entrega'] = df_proveedores['plazo_entrega_dias'] > (q3_plazo + 1.5 * iqr_plazo)
df_proveedores['riesgo_calidad'] = df_proveedores['calificacion_proveedor'] < (q1_calif - 1.5 * iqr_calif)
df_proveedores['proveedor_critico'] = df_proveedores['riesgo_entrega'] & df_proveedores['riesgo_calidad']


### Análisis:

La evaluación estadística de los proveedores se realizó utilizando métricas robustas como el z-score y el rango intercuartílico (IQR) para identificar comportamientos atípicos. Si bien la mayoría de los proveedores se mantienen dentro de rangos normales tanto en calificación como en tiempos de entrega, se identificó al proveedor V010 como un caso especial por presentar un plazo de entrega significativamente superior al promedio. Aunque su calificación es alta, el riesgo logístico asociado a su nivel de demora lo convierte en un candidato para revisión o renegociación, especialmente si su portafolio incluye productos de alta rotación o criticidad operativa. La combinación de métricas objetivas y contexto de negocio es clave para tomar decisiones estratégicas en la gestión de proveedores.

# Carga de outputs

In [87]:
top10_rotacion.to_csv('../data/processed/top10_rotacion.csv', index=False)
df_proveedores.to_csv('../data/processed/proveedores_riesgo.csv', index=False)